In [1]:
import pandas as pd
import requests
import numpy as np
import json
pd.options.mode.chained_assignment = None  # default='warn'

N=607
FD=10
TD=11

In [2]:
url = 'https://fantasy.premierleague.com/api/bootstrap-static/'
r = requests.get(url)
json = r.json()

In [3]:
urlf = []
rf = []
jsonf = []

for x in range(1,N):
    urlf.append('https://fantasy.premierleague.com/api/element-summary/{0}/'.format(x))
    rf.append(requests.get(urlf[x-1]))
    jsonf.append(rf[x-1].json())

In [4]:
elements_df = pd.DataFrame(json['elements'])
elements_type_df = pd.DataFrame(json['element_types'])
teams_df = pd.DataFrame(json['teams'])
events_df = pd.DataFrame(json['events'])
phases_df = pd.DataFrame(json['phases'])

fixtures = []

for y in range(1,N):
    fixtures.append(pd.DataFrame(jsonf[y-1]['fixtures']))
    
fpl=elements_df

In [5]:
fpl['position'] = fpl.element_type.map(elements_type_df.set_index('id').singular_name)
fpl['team'] = fpl.team_code.map(teams_df.set_index('code').name)
fpl['team_points'] = fpl.team_code.map(teams_df.set_index('code').points)
fpl['fixture_difficulty'] = fpl.team_code.map(teams_df.set_index('code').strength)
fpl['3day'] = fpl.team_code.map(teams_df.set_index('code').strength)
fpl['a1'] = fpl.team_code.map(teams_df.set_index('code').strength)
fpl['a2'] = fpl.team_code.map(teams_df.set_index('code').strength)
fpl['a3'] = fpl.team_code.map(teams_df.set_index('code').strength)

In [6]:
fpl_slim=fpl[['id', 'code', 'first_name', 'second_name', 'team', 'position', 'now_cost', 'total_points', 
              'form', 'ict_index', 'fixture_difficulty','3day','a1','a2','a3','status']]
fpl_slim = fpl_slim.sort_values('id', ascending=True)

In [7]:
for z in range(1,N):
    fpl_slim.iloc[z-1,FD] = fixtures[z-1].iloc[0,1]
    fpl_slim.iloc[z-1,TD] = fixtures[z-1].iloc[0,1]+fixtures[z-1].iloc[1,1]+fixtures[z-1].iloc[2,1]

In [27]:
A1 = fpl_slim['a1'].astype('float')
A2 = fpl_slim['a2'].astype('float')
A3 = fpl_slim['a3'].astype('float')

a=1
b=1
c=1
d=1.4
e=1
f=1


COST = fpl_slim['now_cost'].astype('float')
POINTS = fpl_slim['total_points'].astype('float')
FORM = fpl_slim['form'].astype('float')
ICT = fpl_slim['ict_index'].astype('float')
FIX = fpl_slim['fixture_difficulty'].astype('float')
TDAY = fpl_slim['3day'].astype('float')


for i in range(0,N):
    A1[i] =(POINTS[i]**a*ICT[i]**b*FORM[i]**c)/(COST[i]**d*FIX[i]**e*TDAY[i]**f)
    A2[i] =0
    A3[i] =0


fpl_slim['a1']=A1
fpl_slim['a2']=A2
fpl_slim['a3']=A3

In [28]:
fpl_slim1=fpl_slim.sort_values('a1',ascending=False)
fpl_forward=fpl_slim1[fpl_slim1.position.eq('Forward')]
fpl_midfielder=fpl_slim1[fpl_slim1.position.eq('Midfielder')]
fpl_defender=fpl_slim1[fpl_slim1.position.eq('Defender')]
fpl_goalkeeper=fpl_slim1[fpl_slim1.position.eq('Goalkeeper')]

In [29]:
g=0
n=0
d=0
m=0
f=0
cost=0
points=0
M=4


fpl_fin = pd.DataFrame(columns=['id', 'code', 'first_name', 'second_name', 'team', 'position', 'now_cost', 'total_points', 
              'form', 'ict_index', 'fixture_difficulty','3day','a1','a2','a3','status'])


for j in range(1,N):

    if n < 16 and fpl_slim1.iloc[j-1,15] == 'a':
        
        if fpl_slim1.iloc[j-1,5] == 'Goalkeeper':
            g = g + 1
            if g < 3:
                fpl_fin = fpl_fin.append(fpl_slim1.loc[fpl_slim1['id'] == fpl_slim1.iloc[j-1,0]])
                n=n+1

        if fpl_slim1.iloc[j-1,5] == 'Defender':
            d = d + 1
            if d < 6:
                fpl_fin = fpl_fin.append(fpl_slim1.loc[fpl_slim1['id'] == fpl_slim1.iloc[j-1,0]])
                n=n+1
        
        if fpl_slim1.iloc[j-1,5] == 'Midfielder':
            m = m + 1
            if m < 6:
                fpl_fin = fpl_fin.append(fpl_slim1.loc[fpl_slim1['id'] == fpl_slim1.iloc[j-1,0]])
                n=n+1
        
        if fpl_slim1.iloc[j-1,5] == 'Forward':
            f = f + 1
            if f < 4:
                fpl_fin = fpl_fin.append(fpl_slim1.loc[fpl_slim1['id'] == fpl_slim1.iloc[j-1,0]])
                n=n+1
                
    if n == 15:
        
        for k in range(1,16):
            cost = cost + fpl_fin.iloc[k-1,6]
            points = points + fpl_fin.iloc[k-1,7]
        print(cost/10)
        print(points)
        print('chk')
        if cost <= 1000:
            break
        if cost > 1000:
            
            cost = 0
            points=0
            n=n-1
            k=0
            fpl_fin.drop(fpl_fin.tail(1).index,inplace=True)    
            if fpl_slim1.iloc[j-1,5] == 'Goalkeeper':
                g = g - 1
            if fpl_slim1.iloc[j-1,5] == 'Defender':
                d = d - 1  
            if fpl_slim1.iloc[j-1,5] == 'Midfielder':
                m = m - 1        
            if fpl_slim1.iloc[j-1,5] == 'Forward':
                f = f - 1
    





103.7
923
chk
104.3
923
chk
103.5
920
chk
104.0
928
chk
104.9
913
chk
103.9
916
chk
104.4
919
chk
103.4
900
chk
103.9
913
chk
104.5
926
chk
103.4
913
chk
103.8
904
chk
103.8
902
chk
104.2
897
chk
103.3
895
chk
104.1
908
chk
103.6
898
chk
104.8
905
chk
104.4
917
chk
103.4
877
chk
104.2
874
chk
103.3
871
chk
103.3
870
chk
102.9
870
chk
103.4
870
chk
102.9
870
chk
102.9
870
chk
103.3
870
chk
103.4
871
chk
102.9
870
chk
102.9
870
chk
102.9
870
chk
102.9
870
chk
102.9
870
chk
102.9
870
chk
102.9
870
chk
103.4
870
chk
103.4
870
chk
102.9
870
chk
103.2
870
chk
102.9
870
chk
103.3
870
chk
103.2
870
chk
103.3
870
chk
103.6
876
chk
103.6
871
chk
103.2
870
chk
103.2
871
chk
102.9
870
chk
102.9
870
chk
102.9
870
chk
102.9
870
chk
102.9
870
chk
103.3
870
chk
103.2
870
chk
103.2
872
chk
103.3
870
chk


In [30]:
fpl_fin.head(15)

,id,code,first_name,second_name,team,position,now_cost,total_points,form,ict_index,fixture_difficulty,3day,a1,a2,a3,status
378,302,141746,Bruno Miguel,Borges Fernandes,Man Utd,Midfielder,110,83,7.5,136.3,2,8,7.354709,0.0,0.0,a
43,37,114283,Jack,Grealish,Aston Villa,Midfielder,78,73,4.3,132.1,2,7,6.647122,0.0,0.0,a
284,202,106617,Patrick,Bamford,Leeds,Forward,63,75,4.2,100.9,2,8,6.011806,0.0,0.0,a
345,275,103025,Riyad,Mahrez,Man City,Midfielder,84,52,7.5,94.0,2,7,5.297530,0.0,0.0,a
198,164,177815,Dominic,Calvert-Lewin,Everton,Forward,79,83,6.5,123.2,4,9,4.070256,0.0,0.0,a
342,272,61366,Kevin,De Bruyne,Man City,Midfielder,119,56,7.2,113.7,2,7,4.068059,0.0,0.0,a
280,198,72222,Mateusz,Klich,Leeds,Midfielder,55,52,4.0,77.7,2,8,3.697061,0.0,0.0,a
254,224,101668,Jamie,Vardy,Leicester,Forward,103,87,7.0,103.1,3,11,2.893276,0.0,0.0,a
295,213,437495,Illan,Meslier,Leeds,Goalkeeper,46,49,5.8,34.6,2,8,2.888812,0.0,0.0,a
132,110,103192,Kurt,Zouma,Chelsea,Defender,57,69,5.2,53.0,3,9,2.452109,0.0,0.0,a


In [31]:
fpl_forward.head(10)

,id,code,first_name,second_name,team,position,now_cost,total_points,form,ict_index,fixture_difficulty,3day,a1,a2,a3,status
284,202,106617,Patrick,Bamford,Leeds,Forward,63,75,4.2,100.9,2,8,6.011806,0.0,0.0,a
198,164,177815,Dominic,Calvert-Lewin,Everton,Forward,79,83,6.5,123.2,4,9,4.070256,0.0,0.0,a
254,224,101668,Jamie,Vardy,Leicester,Forward,103,87,7.0,103.1,3,11,2.893276,0.0,0.0,a
477,377,200439,Che,Adams,Southampton,Forward,60,60,4.8,83.5,3,9,2.886047,0.0,0.0,a
494,388,78830,Harry,Kane,Spurs,Forward,109,107,6.8,145.1,5,12,2.471791,0.0,0.0,a
223,181,96994,Bobby,Decordova-Reid,Fulham,Forward,53,39,5.0,46.5,2,8,2.184647,0.0,0.0,a
429,506,75115,Callum,Wilson,Newcastle,Forward,66,68,4.3,72.4,3,10,2.000936,0.0,0.0,a
61,514,178301,Ollie,Watkins,Aston Villa,Forward,61,51,1.3,83.5,2,7,1.252017,0.0,0.0,a
138,117,165153,Timo,Werner,Chelsea,Forward,94,57,3.5,94.5,3,9,1.206791,0.0,0.0,a
467,366,84939,Danny,Ings,Southampton,Forward,84,60,3.2,79.0,3,9,1.136511,0.0,0.0,a


In [21]:
fpl_midfielder.head(10)

,id,code,first_name,second_name,team,position,now_cost,total_points,form,ict_index,fixture_difficulty,3day,a1,a2,a3,status
378,302,141746,Bruno Miguel,Borges Fernandes,Man Utd,Midfielder,110,83,7.5,136.3,2,8,48.208381,0.0,0.0,a
43,37,114283,Jack,Grealish,Aston Villa,Midfielder,78,73,4.3,132.1,2,7,37.972701,0.0,0.0,a
345,275,103025,Riyad,Mahrez,Man City,Midfielder,84,52,7.5,94.0,2,7,31.173469,0.0,0.0,a
342,272,61366,Kevin,De Bruyne,Man City,Midfielder,119,56,7.2,113.7,2,7,27.517311,0.0,0.0,a
318,254,118748,Mohamed,Salah,Liverpool,Midfielder,123,95,7.0,140.3,4,9,21.070348,0.0,0.0,a
280,198,72222,Mateusz,Klich,Leeds,Midfielder,55,52,4.0,77.7,2,8,18.365455,0.0,0.0,a
66,57,60307,Pascal,Groß,Brighton,Midfielder,58,39,6.2,59.0,2,7,17.569212,0.0,0.0,a
496,390,85971,Heung-Min,Son,Spurs,Midfielder,96,105,7.8,117.9,5,12,16.763906,0.0,0.0,a
567,445,178186,Jarrod,Bowen,West Ham,Midfielder,64,59,5.8,83.5,3,9,16.535706,0.0,0.0,a
169,141,82403,Wilfried,Zaha,Crystal Palace,Midfielder,75,74,5.0,95.6,3,10,15.720889,0.0,0.0,a


In [22]:
fpl_defender.head(10)

,id,code,first_name,second_name,team,position,now_cost,total_points,form,ict_index,fixture_difficulty,3day,a1,a2,a3,status
132,110,103192,Kurt,Zouma,Chelsea,Defender,57,69,5.2,53.0,3,9,12.356335,0.0,0.0,a
144,123,225796,Reece,James,Chelsea,Defender,53,53,5.0,62.9,3,9,11.648148,0.0,0.0,a
146,232,172850,Benjamin,Chilwell,Chelsea,Defender,62,60,4.8,65.8,3,9,11.320430,0.0,0.0,a
52,46,199798,Ezri,Konsa Ngoyo,Aston Villa,Defender,47,49,4.7,32.3,2,7,11.305000,0.0,0.0,a
282,200,87873,Stuart,Dallas,Leeds,Defender,46,40,3.8,50.3,2,8,10.388043,0.0,0.0,a
554,432,40669,Angelo,Ogbonna,West Ham,Defender,49,49,6.8,41.0,3,9,10.325926,0.0,0.0,a
470,369,93100,Jannik,Vestergaard,Southampton,Defender,49,59,4.8,46.5,3,9,9.953741,0.0,0.0,a
476,376,171771,Jan,Bednarek,Southampton,Defender,46,53,6.0,37.3,3,9,9.550242,0.0,0.0,a
557,435,55459,Aaron,Cresswell,West Ham,Defender,52,53,3.8,66.5,3,9,9.539245,0.0,0.0,a
47,41,149484,Tyrone,Mings,Aston Villa,Defender,53,52,3.3,40.4,2,7,9.343181,0.0,0.0,a


In [23]:
fpl_goalkeeper.head(10)

,id,code,first_name,second_name,team,position,now_cost,total_points,form,ict_index,fixture_difficulty,3day,a1,a2,a3,status
295,213,437495,Illan,Meslier,Leeds,Goalkeeper,46,49,5.8,34.6,2,8,13.360489,0.0,0.0,a
33,12,98980,Emiliano,Martínez,Aston Villa,Goalkeeper,48,53,4.3,25.5,2,7,8.647991,0.0,0.0,a
112,96,98747,Nick,Pope,Burnley,Goalkeeper,54,53,6.0,29.3,3,9,6.390535,0.0,0.0,a
553,431,37096,Lukasz,Fabianski,West Ham,Goalkeeper,51,58,4.2,28.9,3,9,5.112593,0.0,0.0,a
464,363,58376,Alex,McCarthy,Southampton,Goalkeeper,46,50,4.2,29.2,3,9,4.937198,0.0,0.0,a
348,278,121160,Ederson,Santana de Moraes,Man City,Goalkeeper,60,43,4.8,17.2,2,7,4.226286,0.0,0.0,a
406,325,59735,Karl,Darlow,Newcastle,Goalkeeper,50,46,3.7,34.1,3,10,3.869213,0.0,0.0,a
247,217,17745,Kasper,Schmeichel,Leicester,Goalkeeper,55,49,3.8,31.8,3,11,3.262347,0.0,0.0,a
238,516,84182,Alphonse,Areola,Fulham,Goalkeeper,45,30,2.2,32.6,2,8,2.988333,0.0,0.0,a
7,8,80201,Bernd,Leno,Arsenal,Goalkeeper,50,43,3.8,25.5,3,10,2.777800,0.0,0.0,a
